In [ ]:
%pylab inline

TODO:
    
**Smoothing of the FT trace**

In [ ]:
import pandas as pd
import pickle
import json
import os
import numpy as np
import seaborn as sns

In [ ]:
#N_TRIALS = 10
TRIALS = range(1,11)

In [ ]:
SAMPLING_RATE = 1000.

In [ ]:
columns = 'fx fy fz tx ty tz'.split()

In [ ]:
NCHANNELS = 16

In [ ]:
FADE_TIME = .5 # conf["FADE_TIME"] in the experimental script

In [ ]:
CHANNEL_TO_G_MAPPING = [ (0,8), (1,9), (2,10), (3,11), (4,12), (5,13) ]
# Each pair (i,j) tells us that to get the corresponding G value, 
# you need to subtract the j-th channel from the i-th channel

In [ ]:
sensmat = np.loadtxt('../../materials/sensor_transf_matrix_FT4714.csv',delimiter=',')

In [ ]:
sensmat

In [ ]:
PREFIX = 'fttest/fttest_1_arc_29_01.15h08m20'

In [ ]:
def extract_Gs(dat):
    dat = np.array(dat)
    nsamp = int(floor(len(dat)/NCHANNELS)) # this will actually be exact
    captured = dat[ :(nsamp*NCHANNELS) ].reshape(nsamp,NCHANNELS)
    Gs = array([ captured[:,i]-captured[:,j] for (i,j) in CHANNEL_TO_G_MAPPING ]).T  # nsamples x 6
    return Gs

In [ ]:
biasrec = '%s__bias_holding.bin'%PREFIX

In [ ]:
b = pickle.load(open(biasrec,'rb'))
biasraw = extract_Gs(b['ft'])
biasG = mean(biasraw[1000:,:],axis=0)   # TODO: take just the non-NA values here for taking the mean

### Let's read the trial information

In [ ]:
trialdat = pd.read_csv('%s_trials.txt'%PREFIX,sep=' ')
trialdat['t.stay']=trialdat['t.go']-FADE_TIME

In [ ]:
trajectories = pickle.load(open('%s_captured.pickle27'%PREFIX,'rb'))

In [ ]:
def get_trial_data(tr):
    """ Get the data for a given trial, both the trial characteristics, trajectory and force trace """
    trj = [ t for t in trajectories if t['trial']==tr ][0]
    #ft  = allft[tr]
    return trialdat[ trialdat['trial']==tr ].iloc[0],trj

In [ ]:
preproc = []
for tr in TRIALS:
    
    trialinfo,capt = get_trial_data(tr)
    
    traj = capt['trajectory'] # a trace of the positions
    
    Gs   = extract_Gs(capt['ft'])
    # Subtract bias
    G = Gs-biasG
    
    # Push through the sensor matrix to get the forces/torques
    Fs = sensmat.dot(G.T) ## TODO: is sensmat the 'right' way around?
    
    tab = pd.DataFrame(Fs.T)
    tab.columns=columns
    
    #traj = pd.DataFrame(traj)
    assert len(traj) == tab.shape[0] # the position record and the FT record should have the same # of samples
    
    xyz = zip(*traj)
    for i,dm in enumerate(['x','y','z']):
        tab[dm]=xyz[i]
        
    tab['fy']=-tab['fy'] # swap the y (sideways) force because the robot counts it different from the FT sensor
    
    tab['trial']=trialinfo['trial']
    direction = 'left' if tab['y'].iloc[0]>0 else 'right'
    tab['direction'] = direction
    tab['t']=arange(tab.shape[0])/SAMPLING_RATE
    
    preproc.append(tab)
    
preproc=pd.concat(preproc)

In [ ]:
preproc

## Determine where you are on the arc

Code here below comes from the notebook `WhereTheHeckAreWeOnTheArc?`.

In [ ]:
# Also grab a parameter file while we are at it
#PREFIX = 'neeraj/neeraj_1_arc_28_01.14h05m34'
params = json.load(open(PREFIX+'_parameters.json'))
# Convert m to cm
for quant in ["ARC_BASE_X","ARC_BASE_Y","ARC_RADIUS_1","ARC_RADIUS_2","RIGHT_ORIGIN","LEFT_ORIGIN","TARGET_RADIUS"]:
    params[quant]=100*array(params[quant])

In [ ]:
def distance_to_arc(x,y,cx,cy,rad,upper):
    """ 
    Distance to an arc of a given radius around a given center point.
    If upper=True, then the arc is the "upper" half of the circle, that is it extends to HIGHER y than cy
    If upper=False, then the arc is the "lower" half of the circle, that is, it extends to LOWER y than cy
    """
    # First, check that we are in the good half of the circle
    goodhalf = (y>cy and upper) or (y<cy and not upper)
    
    if goodhalf:
        dfromcenter= sqrt((x-cx)**2+(y-cy)**2)
        return abs(dfromcenter-rad)

    # else...
    # In this case, the shortest distance to the arc is the shortest
    # distance to the edges of the arc
    dleftedge  = sqrt((x-(cx-rad))**2+(y-cy)**2)
    drightedge = sqrt((x-(cx+rad))**2+(y-cy)**2)
    return min([dleftedge,drightedge])


def halfcircle_progress(x,y,cx,cy,rad,upper):
    """ Compute the progress along a half-circle defined by center cx,cy and radius, upper """
    dx,dy = x-cx,y-cy
    dy = -dy if not upper else dy
    a = np.arctan2(dy,dx)/(2*np.pi)
    if a<-.25: a+=1 # and a>-.25: a+=1
    return a


def arc_progress(x,y):
    """ This tells us, how much 'progress' the subject has made at this current (x,y) position.
    So when they are at the starting point, the progress will be 0 and in the target, the progress will be 1.
    This is assuming that the subject starts at the right start circle and moves to the left.
    """
    
    # Step 1: determine which half-arc (half-circle) the subject is closest to
    arcs = [ 
        (params["ARC_BASE_X"]+params["ARC_RADIUS_2"],params["ARC_BASE_Y"],params["ARC_RADIUS_2"],False),
        (params["ARC_BASE_X"]-params["ARC_RADIUS_1"],params["ARC_BASE_Y"],params["ARC_RADIUS_1"],True)
        # the last argument in the above tuples tells us whether it's an 'upward' half circle or downward.
           ]
    
    ds = [ distance_to_arc(x,y,cx,cy,rad,upper) for (cx,cy,rad,upper) in arcs ]
    curarc = argmin(ds)
    
    # Step 2: for the circle that they are closest to, compute the progress along this half-circle\
    cx,cy,rad,upper = arcs[curarc]
    p = halfcircle_progress(x,y,cx,cy,rad,upper)+.5*curarc
    
    return p

In [ ]:
#trialinfo
preproc['arc.progress'] = [ arc_progress(y,z) for (y,z) in zip(100*preproc['y'],100*preproc['z']) ] 

In [ ]:
dircol = {'left':'blue','right':'red'}
for (tr,dr),ft in preproc.groupby(['trial','direction']):
    plot(ft['arc.progress'],color=dircol[dr])
sns.despine(offset=5)

In [ ]:
fs = ['fx','fy','fz']
directions = ['left','right']

f,axs = subplots(len(fs),len(directions),sharex=True,figsize=(10,8))

for (tr,dr),ft in preproc.groupby(['trial','direction']):
    
    j = directions.index(dr)
    for i,f in enumerate(fs):
        
        ax = axs[i][j]
        ax.plot(ft['arc.progress'],ft[f])
        
        ax.set_title("%s %s"%(f,dr))
        #ax.plot([])

### Now divide the space into sectors and then average the force

In [ ]:
NSECTORS = 16

sectors = linspace(0,1,NSECTORS)


In [ ]:
sectors

In [ ]:
def which_sector(p):
    return argmin([ abs(s-p) for s in sectors ])

In [ ]:
preproc['sector']=[ which_sector(p) for p in preproc['arc.progress'] ]

In [ ]:
preproc

In [ ]:
fsect = preproc.groupby(['sector','direction']).agg({'fx':mean,'fy':mean,'fz':mean}).reset_index()
fsect['arc.progress']= [ sectors[s] for s in fsect['sector']]

In [ ]:
fsect

Now let's convert the arc progress into a position again!

In [ ]:
def progress_to_pos(p):
    """ Figure out the position corresponding to the 'progress' variable. """
    
    def halfcirc_pos(p,arcinfo):
        cx,cy,radius,upper = arcinfo
        a = p*2*pi # the angle so far
        dx,dy = radius*np.cos(a),radius*np.sin(a)
        if not upper:
            dy = -dy
        return cx+dx,cy+dy

     # Step 1: determine which half-arc (half-circle) the subject is closest to
    arcs = [ 
        (params["ARC_BASE_X"]+params["ARC_RADIUS_2"],params["ARC_BASE_Y"],params["ARC_RADIUS_2"],False),
        (params["ARC_BASE_X"]-params["ARC_RADIUS_1"],params["ARC_BASE_Y"],params["ARC_RADIUS_1"],True)
        # the last argument in the above tuples tells us whether it's an 'upward' half circle or downward.
           ]
    if p<.5:
        return halfcirc_pos(p,arcs[0])
    else:
        return halfcirc_pos(p-.5,arcs[1])

In [ ]:
x,y = zip(*[progress_to_pos(p) for p in fsect['arc.progress']])
fsect['x']=x
fsect['y']=y

In [ ]:
fsect

In [ ]:
def draw_arc(ax):
    """ 
    Draw a reference arc in a given plot
    """
    col   = 'black'
    alpha =.3
    lw    = 2

    circangs = linspace(0,2*pi,100)
    
    def arc(center,radius,angs):
        cx,cy=center
        ax.plot((cx+radius*cos(angs)),
                (cy+radius*sin(angs)),color=col,alpha=alpha,lw=lw)
        # What that factor 100 is doing in there? Converting m to cm.

    arc(params["RIGHT_ORIGIN"],params["TARGET_RADIUS"],circangs)
    arc(params["LEFT_ORIGIN"] ,params["TARGET_RADIUS"],circangs)    
    
    arc((params["ARC_BASE_X"]-params["ARC_RADIUS_1"],params["ARC_BASE_Y"]),params["ARC_RADIUS_1"],linspace(0,   pi,100))
    arc((params["ARC_BASE_X"]+params["ARC_RADIUS_2"],params["ARC_BASE_Y"]),params["ARC_RADIUS_2"],linspace(pi,2*pi,100))
    ax.set_aspect('equal')

In [ ]:
fmagnif = .5
directions = ['left','right']

f,axs = subplots(1,2,figsize=(13,5))
for i,ax in enumerate(axs):
    draw_arc(ax)
    ax.set_title(directions[i])

for d,grp in fsect.groupby('direction'):
    ax = axs[directions.index(d)]
    for i,row in grp.iterrows():
        #if True:
        if True:
            ax.plot(row['x'],row['y'],'o',color='black')
            #ax.text(row['x'],row['y'],row['arc.progress'])
            ax.arrow(row['x'],row['y'],
                     fmagnif*row['fy'],
                     fmagnif*row['fz'],head_width=0.05, head_length=0.1)
            
for ax in axs:
    ax.set_aspect('equal')
    ax.set_xlim(-2,2)
    ax.set_ylim(-1.5,1.5)
    
tight_layout()
sns.despine(offset=5)

**NOTE** The horizontal forces have been INVERTED, it is important to check that this is accurate!!